In [1]:
import json
import time
from pathlib import Path

import numpy as np
import qibo
from qibo import hamiltonians, set_backend
from boostvqe.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)

from boostvqe.ansatze import VQE, build_circuit
from boostvqe.utils import apply_dbi_steps, rotate_h_with_vqe


In [2]:

qibo.set_backend("numpy")

# set the path string which define the results
path = "../results/vqe_data/with_params/10q11l/sgd_10q_11l_42/"

# set the target epoch to which apply DBQA and the number of steps
target_epoch = 2000
dbi_steps = 1

# upload system configuration and parameters for all the training
with open(path + "optimization_results.json") as file:
    config = json.load(file)

losses = dict(np.load(path + "energies.npz"))["0"]
params = np.load(path + f"parameters/params_ite{target_epoch}.npy")


# build circuit, hamiltonian and VQE
hamiltonian = hamiltonians.XXZ(nqubits=config["nqubits"], delta=0.5)
circuit = build_circuit(config["nqubits"], config["nlayers"], "numpy")
vqe = VQE(circuit, hamiltonian)
zero_state = hamiltonian.backend.zero_state(config["nqubits"])
zero_state_t = np.transpose([zero_state])
target_energy = np.min(hamiltonian.eigenvalues())


# set target parameters into the VQE
vqe.circuit.set_parameters(params)
vqe_state = vqe.circuit().state()

ene1 = hamiltonian.expectation(vqe_state)


[Qibo 0.2.9|INFO|2024-06-11 23:10:51]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


[Qibo 0.2.9|INFO|2024-06-11 23:10:53]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


In the VQE numpy


In [3]:
print(vqe.circuit.draw())

q0:     ─RY─RZ─o─────────RY─RZ─────────o─RY─RZ─o─────────RY─RZ─────────o─RY─RZ ...
q1:     ─RY─RZ─Z─────────RY─RZ─o───────|─RY─RZ─Z─────────RY─RZ─o───────|─RY─RZ ...
q2:     ─RY─RZ───o───────RY─RZ─Z───────|─RY─RZ───o───────RY─RZ─Z───────|─RY─RZ ...
q3:     ─RY─RZ───Z───────RY─RZ───o─────|─RY─RZ───Z───────RY─RZ───o─────|─RY─RZ ...
q4:     ─RY─RZ─────o─────RY─RZ───Z─────|─RY─RZ─────o─────RY─RZ───Z─────|─RY─RZ ...
q5:     ─RY─RZ─────Z─────RY─RZ─────o───|─RY─RZ─────Z─────RY─RZ─────o───|─RY─RZ ...
q6:     ─RY─RZ───────o───RY─RZ─────Z───|─RY─RZ───────o───RY─RZ─────Z───|─RY─RZ ...
q7:     ─RY─RZ───────Z───RY─RZ───────o─|─RY─RZ───────Z───RY─RZ───────o─|─RY─RZ ...
q8:     ─RY─RZ─────────o─RY─RZ───────Z─|─RY─RZ─────────o─RY─RZ───────Z─|─RY─RZ ...
q9:     ─RY─RZ─────────Z─RY─RZ─────────Z─RY─RZ─────────Z─RY─RZ─────────Z─RY─RZ ...

q0: ... ─o─────────RY─RZ─────────o─RY─RZ─o─────────RY─RZ─────────o─RY─RZ─o──── ...
q1: ... ─Z─────────RY─RZ─o───────|─RY─RZ─Z─────────RY─RZ─o───────|─RY─RZ─Z──── ...
q2:

# This is the scenario that we can try to compile
$D= \sum_i B_i Z_i$
and Group commutator

In [4]:
from qibo import symbols, hamiltonians
b_list = [1+np.sin(x) for x in np.linspace(-1,1,config["nqubits"]) ]
#b_list = np.random.rand(config["nqubits"])
d = hamiltonians.SymbolicHamiltonian( sum([b*symbols.Z(j) for j,b in zip(range(config["nqubits"]),b_list)]))
from copy import deepcopy
step = 0.12
u2_circ = deepcopy(d).circuit( -step * (np.sqrt(5) - 1) / 2 ) 
u4_circ = deepcopy(d).circuit( step * (np.sqrt(5) + 1) / 2 )
u6_circ = deepcopy(d).circuit( -step  ) 

from boostvqe.compiling_XXZ import *
vqe_circ = vqe.circuit
vqe_circ_inv = vqe_circ.invert()
nmb_ts_steps = 1
u1_circ = vqe_circ +nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=-step * (np.sqrt(5) - 1) / 2,delta=0.5,steps=nmb_ts_steps, order = 2)+vqe_circ_inv 
u3_circ = vqe_circ +nqubit_XXZ_decomposition(nqubits=config["nqubits"],t= step ,delta=0.5,steps=nmb_ts_steps, order = 2)+vqe_circ_inv 
u5_circ = vqe_circ +nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=-step * (3 - np.sqrt(5)) / 2,delta=0.5,steps=nmb_ts_steps, order = 2)+vqe_circ_inv 



circ_boost = (  u6_circ + u5_circ + u4_circ + u3_circ + u2_circ +vqe_circ )

ket_boost = circ_boost(initial_state=zero_state)
hamiltonian.expectation(ket_boost.state()) 

-15.032761300698297

In [5]:
t=0
for g in circ_boost.queue:
    if isinstance(g, qibo.gates.gates.CNOT):
        t = t +1
t

90

In [6]:
t=0
for g in circ_boost.queue:
    if isinstance(g, qibo.gates.gates.CZ):
        t = t +1
t

550

In [7]:
t=0
for g in vqe.circuit.queue:
    if isinstance(g, qibo.gates.gates.CZ):
        t = t +1
t

110

In [8]:
1050/150

7.0

In [9]:
np.sqrt(5)-3

-0.7639320225002102

In [10]:
ene1

-14.938487070478573

In [11]:
target_energy

-15.276131122065795

In [12]:
from boostvqe.models.dbi.group_commutator_iteration_transpiler import *
eo_d = EvolutionOracle(d,mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation,name = "D(linear)")

hamiltonian.delta = 0.5
from boostvqe.models.dbi.double_bracket_evolution_oracles import *
# quick hack how to use the method function hamiltonians.XXZ to get a simulable oracle
eo_xxz = XXZ_EvolutionOracle(hamiltonian,mode_evolution_oracle=EvolutionOracleType.numerical, steps = 1, order = 2)
eo_xxz.mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation
hamiltonian.dense = hamiltonian
# prescribe nmb of steps
eo_d.please_use_prescribed_nmb_ts_steps = 1
eo_xxz.please_use_prescribed_nmb_ts_steps = 1


gci  = GroupCommutatorIterationWithEvolutionOracles(input_hamiltonian_evolution_oracle=
                            FrameShiftedEvolutionOracle(eo_xxz, before_circuit=vqe.circuit.invert(), 
                                                        after_circuit=vqe.circuit,name="shifting by vqe"), 
                                                        mode_double_bracket_rotation=DoubleBracketRotationType.group_commutator_third_order_reduced)
gci(step**2, eo_d)
hamiltonian.expectation(gci.iterated_hamiltonian_evolution_oracle.get_composed_circuit()().state())

-15.032761300698017

In [13]:
a = deepcopy(gci.iterated_hamiltonian_evolution_oracle.get_composed_circuit())
gci2  = GroupCommutatorIterationWithEvolutionOracles(input_hamiltonian_evolution_oracle=
                            FrameShiftedEvolutionOracle(eo_xxz, before_circuit=a.invert(), 
                                                        after_circuit=a,name="shifting by vqe"), 
                                                        mode_double_bracket_rotation=DoubleBracketRotationType.group_commutator_third_order_reduced)

In [14]:
dd = hamiltonians.SymbolicHamiltonian( sum([1.2*j*symbols.Z(j) for j in range(10)]))
eo_dd = EvolutionOracle(dd,mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation,name = "D(linear)")
for s in np.linspace(0.01,0.02,6):
    circ_update = gci2.group_commutator(s,eo_dd)["forwards"]+ a
    print(hamiltonian.expectation(circ_update().state()))


[Qibo 0.2.9|WARNING|2024-06-11 23:11:05]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.9|WARNING|2024-06-11 23:16:22]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [ ]:
dd = hamiltonians.SymbolicHamiltonian( sum([symbols.Z(j) for j in range(10)]))
eo_dd = EvolutionOracle(dd,mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation,name = "D(linear)")

loss_list = []
for s in np.linspace(0,0.1,20):
    circ_update = gci2.group_commutator(s,eo_d)["forwards"]+ a
    loss_list.append(hamiltonian.expectation(circ_update().state()))
                     
import matplotlib.pyplot as plt 
plt.plot(loss_list)

AttributeError: 'numpy.int64' object has no attribute 'init_kwargs'

In [ ]:

a = np.argmin(loss_list)
times = np.linspace(0,0.2,92)
times[a]

0.004395604395604396

In [ ]:
loss_list[a]

-14.811148950309335

In [ ]:
ds

In [ ]:
gci(times[a],eo_d)

In [ ]:
dd = hamiltonians.SymbolicHamiltonian( sum([symbols.Z(j) for j in range(10)]))
eo_dd = EvolutionOracle(dd,mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation,name = "D(linear)")
loss_list = []
for s in np.linspace(0,0.1,2):
    circ_update = gci.group_commutator(s,eo_dd)["forwards"]+ gci.iterated_hamiltonian_evolution_oracle.get_composed_circuit()
    loss_list.append(hamiltonian.expectation(circ_update().state()))
                     
import matplotlib.pyplot as plt 
plt.plot(loss_list)

[Qibo 0.2.9|WARNING|2024-06-11 21:21:38]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


KeyboardInterrupt: 

In [ ]:
a = np.argmin(loss_list)
times = np.linspace(0,0.2,92)
times[a]

0.0

In [ ]:
adm = d.dense.matrix
v_vqe = vqe.circuit.unitary()
h_vqe = v_vqe.conj().T @ hamiltonian.matrix @ v_vqe
h_vqe_ham = hamiltonians.Hamiltonian(nqubits = config["nqubits"],matrix=h_vqe)

from copy import deepcopy
dbi = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.group_commutator_3,
)

u1 = dbi.h.exp(-step * (np.sqrt(5) - 1) / 2)
u2 = dbi.backend.calculate_matrix_exp(-step * (np.sqrt(5) - 1) / 2, dm)
u3 = dbi.h.exp(step)
u4 = dbi.backend.calculate_matrix_exp(step * (np.sqrt(5) + 1) / 2, dm)
u5 = dbi.h.exp(-step * (3 - np.sqrt(5)) / 2)
u6 = dbi.backend.calculate_matrix_exp(-step, dm)

u_gc3_by_hand = u1 @ u2 @ u3 @ u4 @ u5 @ u6
h_gc3_by_hand = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand = h_gc3_by_hand.expectation(zero_state_t)
print(ene1)
print(energy_dbi_gc3_by_hand)

[Qibo 0.2.9|WARNING|2024-06-10 22:27:45]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


-14.560174594055583
-14.752268266791006


In [ ]:
print(np.linalg.norm( u1-vw_xxz_compiling_phase(config["nqubits"],nmb_ts_steps,order = 2)*( u1_circ ).unitary() ))

0.006991213069987698


In [ ]:
print(np.linalg.norm( u2-( u2_circ ).unitary() ))
print(np.linalg.norm( u3-vw_xxz_compiling_phase(config["nqubits"],nmb_ts_steps,order = 2)*( u3_circ ).unitary() ))
print(np.linalg.norm( u4-( u4_circ ).unitary() ))
print(np.linalg.norm( u5-vw_xxz_compiling_phase(config["nqubits"],nmb_ts_steps,order = 2)*( u5_circ ).unitary() ))
print(np.linalg.norm( u6-( u6_circ ).unitary() ))

1.107953320613948e-14
0.029094422053674126
28.1467227947752
0.0016616352501537186
5.311895747628695


In [ ]:
u_gc3_by_hand2 = u1 @ u2 @ u3 @ u4 @ u5_circ.unitary() @ u6_circ.unitary()
h_gc3_by_hand2 = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand2.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand2, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand2 = h_gc3_by_hand2.expectation(zero_state_t)
print(ene1)
print(energy_dbi_gc3_by_hand2)

-14.560174594055583
-14.752263119810468


In [ ]:
u_gc3_by_hand2 = u1 @ u2 @ u3 @ u4_circ.unitary() @ u5_circ.unitary() @ u6_circ.unitary()
h_gc3_by_hand2 = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand2.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand2, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand2 = h_gc3_by_hand2.expectation(zero_state_t)
print(ene1)
print(energy_dbi_gc3_by_hand2)

-14.560174594055583
-14.718760002748295


In [ ]:
u_gc3_by_hand2 = u1 @ u2 @ u3_circ.unitary() @ u4_circ.unitary() @ u5_circ.unitary() @ u6_circ.unitary()
h_gc3_by_hand2 = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand2.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand2, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand2 = h_gc3_by_hand2.expectation(zero_state_t)
print(ene1)
print(energy_dbi_gc3_by_hand2)

-14.560174594055583
-14.560174594055697


In [ ]:
u_gc3_by_hand2 = u1 @ u2_circ.unitary() @ u3_circ.unitary() @ u4_circ.unitary() @ u5_circ.unitary() @ u6_circ.unitary()
h_gc3_by_hand2 = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand2.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand2, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand2 = h_gc3_by_hand2.expectation(zero_state_t)
print(ene1)
print(energy_dbi_gc3_by_hand2)

-14.560174594055583
-14.560174594055667


In [ ]:
u_gc3_by_hand2 = u1 @ u2_circ.unitary() @ u3_circ.unitary() @ u4_circ.unitary() @ u5 @ u6_circ.unitary()
h_gc3_by_hand2 = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand2.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand2, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand2 = h_gc3_by_hand2.expectation(zero_state_t)
print(ene1)
print(energy_dbi_gc3_by_hand2)

-14.560174594055583
-14.621201272487871


In [ ]:
u_gc3_by_hand3 = (u6_circ+u5_circ+u4_circ+u3_circ+u2_circ+u1_circ).unitary()
h_gc3_by_hand3 = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand3.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand3, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand3 = h_gc3_by_hand3.expectation(zero_state_t)
print(ene1)

print(energy_dbi_gc3_by_hand3)

-14.560174594055583
-14.560174594055773


In [ ]:
u_gc3_by_hand4 = vqe.circuit.unitary() @ u_gc3_by_hand3
h_gc3_by_hand4 = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand4.conj().T @ deepcopy(hamiltonian).matrix @ u_gc3_by_hand4, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand4 = h_gc3_by_hand4.expectation(zero_state_t)
print(ene1)
print(energy_dbi_gc3_by_hand4)

-14.560174594055583
-14.560174594055773


In [ ]:
blabla to stop 

SyntaxError: invalid syntax (3556992881.py, line 1)

In [ ]:
dm = d.dense.matrix
v_vqe = vqe.circuit.unitary()
h_vqe = v_vqe.conj().T @ hamiltonian.matrix @ v_vqe
h_vqe_ham = hamiltonians.Hamiltonian(nqubits = config["nqubits"],matrix=h_vqe)

from copy import deepcopy
dbi = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.group_commutator_3,
)


step = 0.12

dbi(0.12,d=dm)
energy_dbi_gc3 = dbi.h.expectation(zero_state_t)

[Qibo 0.2.9|WARNING|2024-06-10 18:29:47]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [ ]:
from copy import deepcopy
dbi2 = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.group_commutator_3_reduced,
)


step = 0.12

dbi2(0.12,d=dm)
energy_dbi_gc32 = dbi2.h.expectation(zero_state_t)

In [ ]:
energy_dbi_gc3

-14.752268266791006

In [ ]:
energy_dbi_gc32

-14.752268266790963

In [ ]:
dbi = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.group_commutator_3,
)
u_gc3 = dbi.eval_dbr_unitary(step,d=dm)

In [ ]:
circ_boost = (
    vqe_circ + v1_circ +vqe_circ_inv 
    + vd1_circ 
    + vqe_circ + v2_circ +vqe_circ_inv 
    + vd3_circ
    + vqe_circ + v1_circ +vqe_circ_inv 
    + vd3_circ
    + vqe_circ 
)

In [ ]:
np.linalg.norm(u_gc3 - circ_boost.unitary()* np.exp(1j *np.pi/4*880))

KeyboardInterrupt: 

# Alternate way is much slower

In [ ]:
dm = d.dense.matrix

[Qibo 0.2.9|WARNING|2024-06-07 17:32:13]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [ ]:
v_vqe = vqe.circuit.unitary()

In [ ]:
h_vqe = v_vqe.conj().T @ hamiltonian.matrix @ v_vqe
h_vqe_ham = hamiltonians.Hamiltonian(nqubits = config["nqubits"],matrix=h_vqe)

In [ ]:
from copy import deepcopy
dbi = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.group_commutator_3,
)
import inspect
inspect.getfile(dbi.__class__)

step = 0.12

u_gc3 = dbi.eval_dbr_unitary(step,d=dm)
h_gc3_exposed = hamiltonians.Hamiltonian(matrix = u_gc3.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3, nqubits= config["nqubits"] )

dbi(0.12,d=dm)
energy_dbi_gc3 = dbi.h.expectation(zero_state_t)
energy_dbi_gc3_exposed = h_gc3_exposed.expectation(zero_state_t)

In [ ]:
u1 = dbi.h.exp(-step * (np.sqrt(5) - 1) / 2)
u2 = dbi.backend.calculate_matrix_exp(-step * (np.sqrt(5) - 1) / 2, dm)
u3 = dbi.h.exp(step)
u4 = dbi.backend.calculate_matrix_exp(step * (np.sqrt(5) + 1) / 2, dm)
u5 = dbi.h.exp(-step * (3 - np.sqrt(5)) / 2)
u6 = dbi.backend.calculate_matrix_exp(-step, dm)

u_gc3_by_hand = u1 @ u2 @ u3 @ u4 @ u5 @ u6
h_gc3_by_hand = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand, nqubits= config["nqubits"] )

energy_dbi_gc3_by_hand = h_gc3_by_hand.expectation(zero_state_t)
print(ene1)
print(energy_dbi_gc3)

# Runs too long to bother to wait when casting to numpy

In [ ]:
v1 = v_vqe.conj().T @ v1_circ.unitary() @ v_vqe
v2 = v_vqe.conj().T @ v2_circ.unitary()@ v_vqe
v3 = v_vqe.conj().T @ v3_circ.unitary()@ v_vqe

u_gc3_by_hand_from_circ = v1 @ vd1_circ.unitary() @ v2 @ vd2_circ.unitary() @ v3 @ vd3_circ.unitary()
h_gc3_by_hand_from_circ = hamiltonians.Hamiltonian(
    matrix = u_gc3_by_hand_from_circ.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand_from_circ )
energy_dbi_gc3_by_hand = h_gc3_by_hand_from_circ.expectation(zero_state_t)
print(energy_dbi_gc3_by_hand)

KeyboardInterrupt: 

In [ ]:
c = Circuit(nqubits=2)
c.add(gates.RZ(0, theta = step))
c.add(gates.RZ(1, theta = 1.2*step))
ddd = hamiltonians.SymbolicHamiltonian(symbols.Z(0)+1.2*symbols.Z(1), nqubits=2)
np.linalg.norm(ddd.exp(step/2) - c.unitary())

[Qibo 0.2.9|WARNING|2024-06-07 17:57:27]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


4.9065389333867974e-18

In [ ]:
np.linalg.norm(d.exp(1)-d.circuit(1).unitary())

[Qibo 0.2.9|WARNING|2024-06-10 18:56:18]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


1.8075301951460534e-14

In [ ]:
        
for t in np.linspace(.1,.14,5):
    for mode  in [DoubleBracketGeneratorType.group_commutator_3]:        
        print (t)
        dbi(t,d=dm, mode = mode)
        print(dbi.h.expectation(zero_state_t)-ene1)
        dbi.h = deepcopy(h_vqe_ham)

0.1
-0.17370846444809196
0.11000000000000001
-0.1858758485760017
0.12000000000000001
-0.19209367273542632
0.13
-0.19163474209593012
0.14
-0.18406164639815614


-14.751809336151513

In [ ]:
_ - ene1

-0.17370846444809196

In [ ]:
u_gc3_CNOT = v1 @ u2 @ v2  @ u4 @ v3 @ u6


In [ ]:

dbi2 = DoubleBracketIteration(
    hamiltonian=hamiltonians.Hamiltonian(matrix = u_gc3_CNOT.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_CNOT, nqubits = config["nqubits"]),
    mode=DoubleBracketGeneratorType.single_commutator,
)

In [ ]:
dbi2.h.expectation(zero_state_t)

-14.74579723717103

In [ ]:
_ - ene1

-0.18562264311544752

$\langle 0| V_1\dagger U_\theta^\dagger  H_0 U_\theta V_1 |0\rangle$

$|dbqa \rangle = U_\theta V_1 |0\rangle$

In [ ]:
        
for t in np.linspace(.001,.1,3):
    for mode  in [DoubleBracketGeneratorType.single_commutator, DoubleBracketGeneratorType.group_commutator_3]:
        
        print (t)
        dbi(t,d=dm, mode = mode)
        print(dbi.h.expectation(zero_state_t)-ene1)
        dbi.h = deepcopy(h_vqe_ham)

0.001
0.03049416970667984
0.001
-2.93783569489392e-05
0.0505
3.1454674430590615
0.0505
-0.06606296647770904
0.1
7.074245955138497
0.1
-0.17370846444809196


In [ ]:
        
for t in np.linspace(.001,.1,13):
    for mode  in [DoubleBracketGeneratorType.single_commutator, DoubleBracketGeneratorType.group_commutator_3]:
        
        print (t)
        dbi(t,d=-dm, mode = mode)
        print(dbi.h.expectation(zero_state_t)-ene1)
        dbi.h = deepcopy(h_vqe_ham)

0.001
-0.02824536086750129
0.001
2.9380340411222505e-05
0.009250000000000001
-0.1680169705742678
0.009250000000000001
0.0025178786739274983
0.0175
-0.12216196003698343
0.0175
0.00904829844227173
0.025750000000000002
0.12697893245887215
0.025750000000000002
0.019708017902930308
0.034
0.5809235629930072
0.034
0.03461163575132886
0.04225
1.2225916652057816
0.04225
0.0538671984955581
0.0505
2.0170847723539627
0.0505
0.07753647566399202
0.058750000000000004
2.9158127725243492
0.058750000000000004
0.1055948106119633
0.067
3.863256246756647
0.067
0.13789621831791443
0.07525000000000001
4.804975983893996
0.07525000000000001
0.1741487964359827
0.0835
5.695149857470556
0.0835
0.2139042537835767
0.09175
6.502038526684503
0.09175
0.2565636208389197
0.1
7.210326149454998
0.1
0.30139922935152264


## We need to schedule unitaries such as this one below

This later needs to be generalized to the GC3 PR from Andrew to get the same performance



In [ ]:
def u_gcr(h,d,t):
    ud =  d.exp(t)
    u = h.exp(-t)
    return     ud.conj().T @ u @ud


In [ ]:
for t in np.linspace(1e-9,1e-6,3):
    u_gcrd = u_gcr(h_vqe_ham,d,t)
    print(h_vqe_ham.expectation(u_gcrd@zero_state_t)-ene1)

5.329070518200751e-15
3.963354089364657e-10
1.5823502508283127e-09


- This doesn't work now. It's better to have the correct expose dbr unitary and then take the exposed unitaries and compare to compiled circuits.


If someone picks it up:
- please try to find the correct scheduling as above
- then expose the unitaries
- and then use the XXZ decomposition function to replace h.exp(-t) by a circuit